In [1]:
import socket
import threading
import time
from queue import Queue
from datetime import datetime

In [2]:
THREADS = 3
JOB_ID = [1,2,3]

queue = Queue()

all_ips = []
all_addresses = []

server_address = ("192.168.43.223",3000)

lock_active = 0

bufferSize = 64

In [3]:
msgFromAdmin = "Master here!"

msgDrop = "Drop all!"

msgRefresh = "Update!"

msgStatus = "Status!"
msgStatusToBytes = str.encode(msgStatus)

msgLockNotOnline = "Lock Not Online!"
msgLockNotOnlineToBytes = str.encode(msgLockNotOnline)

msgFromServer = "Hello from server"
msgToBytes = str.encode(msgFromServer)

msgLocked = "Zakljucano"
msgLockedToBytes = str.encode(msgLocked)

msgUnlocked = "Otkljucano"
msgUnlockedToBytes = str.encode(msgUnlocked)

In [4]:
# Create datagram socket

serverSocket = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)

serverSocket.bind(server_address)

print("UDP Server is up and listening")

UDP Server is up and listening


In [5]:
# Handling connections

def connections_handler():
    
    # booleans for lock initialization
    global lock_address, lock_active, lock_ip
    
    # delete all addresses from last time server was active
    del all_addresses[:]
    
    while True:
        
        dateTimeObj = datetime.now()
        # Receive message and client IP and port
        message, address = serverSocket.recvfrom(bufferSize)
        # Store client IP and port tuple in a list
        all_addresses.append(address)
        all_ips.append(address[0])
        
        # Check if new incoming client is connected to the server
        if (all_ips.count(address[0]) == 1 ):
                        
            print(f"{dateTimeObj.hour}:{dateTimeObj.minute}:{dateTimeObj.second}  New connection added ==> {address[0]}\n")
        
        
        
        if (all_ips.count(address[0]) == 2):
            for addr in all_addresses:
                if(addr[0] == address[0]):
                    all_addresses.remove(addr)
                    break
        
        #print(message)
        if (message == b'Master here!'):
            admin_ip = address[0]
            admin_address = address
            
            print(f"Admin locked in at IP Address {admin_ip}!!\n")
            serverSocket.sendto(msgToBytes,admin_address)
        
        
        if(message == b'Drop all!'):
            for ip in all_addresses:
                if(ip != admin_address):
                    all_addresses.remove(ip)
            print("Dropped all users!\n")
        
        
        if(message == b'Update!'):
            if (lock_active == 1 ):
                serverSocket.sendto(msgStatusToBytes, lock_ip)
            else:
                serverSocket.sendto(msgLockNotOnlineToBytes, admin_address)
        
        if (message == b'Your lock, reporting for duty!'):
            
            lock_ip = address[0]
            lock_address = address
            
            print(f"Got your lock locked in at IP Address {lock_ip}!!\n")
            lock_active = 1
            
            serverSocket.sendto(msgToBytes, lock_address)
            
        
        if (lock_active == 0):
            
            print("We are waiting for lock to come online, please wait....")
            if(message == b'Hello, client here!\n'):
                serverSocket.sendto(msgToBytes, address)
        
        if (lock_active == 1):
            
            if( address[0] == lock_ip ):
                #print(message)
                if(message == b'Locked'):
                    print(f"{dateTimeObj.hour}:{dateTimeObj.minute}:{dateTimeObj.second} => LOCKED\n")
                    for ip in all_addresses:
                        if(ip != lock_address):
                            serverSocket.sendto(msgLockedToBytes, ip)

                elif(message == b'Unlocked'):
                    print(f"{dateTimeObj.hour}:{dateTimeObj.minute}:{dateTimeObj.second} => UNLOCKED\n")
                    for ip in all_addresses:
                        if(ip != lock_address):
                            serverSocket.sendto(msgUnlockedToBytes, ip)
                
                elif(message == b'Lock says: Listening...'):
                    print("Lock active and ready to go!\n")
        
                            
            else:
                if(message == b'LOCK!'):
                    for ip in all_addresses:
                        serverSocket.sendto(msgLockedToBytes, ip)
                        
                elif(message == b'UNLOCK!'):
                    for ip in all_addresses:
                        serverSocket.sendto(msgUnlockedToBytes, ip)
                        
                elif(message == b'Hello, client here!'):
                    serverSocket.sendto(msgToBytes, address)


In [ ]:
connections_handler()

21:31:58  New connection added ==> 192.168.43.251

Got your lock locked in at IP Address 192.168.43.251!!

Lock active and ready to go!

21:32:39  New connection added ==> 192.168.43.1

Admin locked in at IP Address 192.168.43.1!!

21:33:15 => UNLOCKED

21:33:20 => LOCKED

21:33:33 => UNLOCKED

21:33:38 => LOCKED

21:33:41 => UNLOCKED

21:33:42 => LOCKED

21:34:11 => UNLOCKED

21:34:52 => LOCKED

21:34:54 => UNLOCKED

21:34:56 => LOCKED

21:43:54 => UNLOCKED

21:44:4 => UNLOCKED

21:44:7 => LOCKED

21:44:52 => UNLOCKED

21:44:57 => LOCKED

21:45:0 => UNLOCKED

21:45:7 => LOCKED

21:45:17 => UNLOCKED

21:45:45 => LOCKED

21:45:45  New connection added ==> 192.168.43.55

